# Milestone 05: 銀行帳戶系統 - 起始程式碼

## 📋 使用說明

這個 notebook 提供完整的類別框架與方法模板，協助您開始實作銀行帳戶系統。每個類別都包含：

- **完整的類別結構**：所有必要的屬性與方法
- **方法簽名**：正確的參數與回傳值型態
- **實作提示**：TODO 註解指引實作方向
- **範例程式碼**：部分關鍵邏輯的參考實作

## 🎯 實作順序建議

1. **Transaction** → 最基礎的資料類別
2. **Customer** → 客戶資訊管理
3. **BankAccount** → 帳戶基礎類別
4. **繼承類別** → SavingsAccount, CheckingAccount, TimeDepositAccount
5. **Bank** → 銀行系統整合
6. **進階功能** → CreditAccount, 特殊方法等

---

## 📦 匯入模組與型態提示

In [ ]:
from typing import List, Dict, Optional, Union
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
import re
from functools import lru_cache

## 🚨 自訂例外類別

In [ ]:
# TODO: 實作完整的例外類別架構
# 提示：建立階層式的例外處理系統

class BankException(Exception):
    """銀行系統基礎例外類別"""
    pass

class AccountException(BankException):
    """帳戶操作例外"""
    pass

class InsufficientFundsException(AccountException):
    """餘額不足例外"""
    def __init__(self, available_balance: float, requested_amount: float):
        self.available_balance = available_balance
        self.requested_amount = requested_amount
        # TODO: 實作清晰的錯誤訊息
        super().__init__(f"餘額不足：可用餘額 ${available_balance:.2f}，要求金額 ${requested_amount:.2f}")

class AccountClosedException(AccountException):
    """帳戶已關閉例外"""
    def __init__(self, account_number: str):
        # TODO: 實作帳戶關閉的錯誤訊息
        pass

class InvalidAmountException(AccountException):
    """無效金額例外"""
    def __init__(self, amount: float):
        # TODO: 實作金額驗證的錯誤訊息
        pass

class WithdrawalLimitExceededException(AccountException):
    """提款次數超限例外"""
    def __init__(self, current_count: int, limit: int):
        # TODO: 實作提款限制的錯誤訊息
        pass

class CustomerException(BankException):
    """客戶操作例外"""
    pass

class CustomerNotFoundException(CustomerException):
    """客戶未找到例外"""
    def __init__(self, customer_id: str):
        # TODO: 實作客戶查詢的錯誤訊息
        pass

class DuplicateCustomerException(CustomerException):
    """客戶重複例外"""
    def __init__(self, customer_id: str):
        # TODO: 實作客戶重複的錯誤訊息
        pass

class TransferException(BankException):
    """轉帳操作例外"""
    pass

class SameAccountTransferException(TransferException):
    """同帳戶轉帳例外"""
    def __init__(self, account_number: str):
        # TODO: 實作同帳戶轉帳的錯誤訊息
        pass

## 🛠️ 資料驗證工具函式

In [ ]:
def validate_amount(amount: float) -> float:
    """驗證交易金額
    
    Args:
        amount: 要驗證的金額
    
    Returns:
        float: 驗證後的金額（四捨五入到小數點後兩位）
    
    Raises:
        InvalidAmountException: 金額無效時拋出
    """
    # TODO: 實作金額驗證邏輯
    # 提示：檢查型態、正數、上限、小數位數
    if not isinstance(amount, (int, float)):
        raise InvalidAmountException(amount)
    
    # TODO: 加入更多驗證邏輯
    # - 金額必須 > 0
    # - 金額不能超過 $1,000,000
    # - 四捨五入到小數點後兩位
    
    return round(amount, 2)

def validate_account_number(account_number: str) -> str:
    """驗證帳戶號碼格式"""
    # TODO: 實作帳戶號碼驗證
    # 提示：6位數字，以1開頭
    pass

def validate_email(email: str) -> str:
    """驗證電子郵件格式"""
    # TODO: 使用正規表達式驗證 email 格式
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    # TODO: 完成驗證邏輯
    return email.lower()

def validate_phone(phone: str) -> str:
    """驗證電話號碼格式"""
    # TODO: 實作電話號碼驗證（台灣手機格式）
    # 提示：09XX-XXX-XXX 或 09XXXXXXXX
    pass

## 💳 Transaction 交易記錄類別

In [ ]:
class Transaction:
    """交易記錄類別
    
    記錄銀行帳戶的所有交易資訊，包括存款、提款、轉帳等。
    提供完整的交易追蹤與查詢功能。
    """
    
    # 類別變數：交易序號計數器
    _transaction_counter = 1
    
    def __init__(self, transaction_type: str, amount: float, 
                 description: str = "", balance_after: float = 0.0):
        """初始化交易記錄
        
        Args:
            transaction_type: 交易類型 ('deposit', 'withdraw', 'transfer_in', 'transfer_out')
            amount: 交易金額
            description: 交易描述
            balance_after: 交易後帳戶餘額
        """
        # TODO: 實作交易記錄初始化
        # 提示：自動生成交易ID、設定時間戳記
        self._transaction_id = f"T{self._transaction_counter:06d}"
        Transaction._transaction_counter += 1
        
        self._transaction_type = transaction_type
        self._amount = validate_amount(amount)
        self._timestamp = datetime.now()
        self._description = description
        self._balance_after = balance_after
    
    @property
    def transaction_id(self) -> str:
        """取得交易ID（唯讀）"""
        return self._transaction_id
    
    @property
    def transaction_type(self) -> str:
        """取得交易類型（唯讀）"""
        return self._transaction_type
    
    @property
    def amount(self) -> float:
        """取得交易金額（唯讀）"""
        return self._amount
    
    @property
    def timestamp(self) -> datetime:
        """取得交易時間（唯讀）"""
        return self._timestamp
    
    def __str__(self) -> str:
        """回傳友善的交易記錄字串
        
        Returns:
            str: 格式化的交易記錄
        
        Example:
            "2024-01-15 14:30:25 | 存款 | +$500.00 | 餘額: $1,500.00"
        """
        # TODO: 實作友善的交易記錄格式
        # 提示：包含時間、類型、金額、餘額
        type_map = {
            'deposit': '存款',
            'withdraw': '提款',
            'transfer_in': '轉入',
            'transfer_out': '轉出'
        }
        
        # TODO: 完成格式化邏輯
        pass
    
    def __repr__(self) -> str:
        """回傳物件的詳細表示"""
        # TODO: 實作 repr 方法
        pass
    
    def __eq__(self, other) -> bool:
        """比較兩筆交易是否相同（比較交易ID）"""
        # TODO: 實作交易比較邏輯
        if not isinstance(other, Transaction):
            return False
        return self._transaction_id == other._transaction_id

## 👤 Customer 客戶類別

In [ ]:
class Customer:
    """客戶類別
    
    管理客戶的基本資訊與其擁有的銀行帳戶。
    提供客戶資料維護與帳戶統計功能。
    """
    
    def __init__(self, customer_id: str, name: str, email: str, phone: str):
        """初始化客戶資料
        
        Args:
            customer_id: 客戶編號（唯一識別碼）
            name: 客戶姓名
            email: 電子郵件
            phone: 電話號碼
        """
        # TODO: 實作客戶資料初始化
        # 提示：驗證輸入資料、初始化帳戶列表
        self._customer_id = customer_id
        self._name = name
        self._email = validate_email(email)
        self._phone = validate_phone(phone)
        self._accounts: List['BankAccount'] = []  # 前向參考
        self._created_date = datetime.now()
    
    @property
    def customer_id(self) -> str:
        """取得客戶ID（唯讀）"""
        return self._customer_id
    
    @property
    def name(self) -> str:
        """取得客戶姓名（唯讀）"""
        return self._name
    
    @property
    def email(self) -> str:
        """取得客戶信箱（唯讀）"""
        return self._email
    
    @property
    def phone(self) -> str:
        """取得客戶電話（唯讀）"""
        return self._phone
    
    def add_account(self, account: 'BankAccount') -> None:
        """新增帳戶到客戶名下
        
        Args:
            account: 要新增的銀行帳戶
        
        Raises:
            ValueError: 帳戶已存在時拋出
        """
        # TODO: 實作帳戶新增邏輯
        # 提示：檢查重複、建立雙向關聯
        if account in self._accounts:
            raise ValueError(f"帳戶 {account.account_number} 已存在於客戶名下")
        
        # TODO: 完成新增邏輯
        pass
    
    def remove_account(self, account_number: str) -> bool:
        """移除指定帳戶
        
        Args:
            account_number: 要移除的帳戶號碼
        
        Returns:
            bool: 移除是否成功
        """
        # TODO: 實作帳戶移除邏輯
        # 提示：找到帳戶後移除，注意處理不存在的情況
        pass
    
    def get_total_balance(self) -> float:
        """計算所有帳戶總餘額
        
        Returns:
            float: 所有帳戶的餘額總和
        """
        # TODO: 實作總餘額計算
        # 提示：遍歷所有帳戶並加總餘額
        pass
    
    def get_accounts_by_type(self, account_type: str) -> List['BankAccount']:
        """取得指定類型的所有帳戶
        
        Args:
            account_type: 帳戶類型名稱
        
        Returns:
            List[BankAccount]: 符合類型的帳戶列表
        """
        # TODO: 實作帳戶類型篩選
        # 提示：使用 isinstance() 檢查類型
        pass
    
    def get_account_count(self) -> int:
        """取得帳戶總數"""
        return len(self._accounts)
    
    def __str__(self) -> str:
        """回傳友善的客戶資訊字串"""
        # TODO: 實作客戶資訊格式化
        # 提示：包含姓名、ID、帳戶數量、總餘額
        pass
    
    def __repr__(self) -> str:
        """回傳物件的詳細表示"""
        return f"Customer(customer_id='{self._customer_id}', name='{self._name}')"

## 🏦 BankAccount 銀行帳戶基礎類別

In [ ]:
class BankAccount(ABC):
    """銀行帳戶抽象基礎類別
    
    定義所有銀行帳戶的共同屬性與行為。
    使用抽象基礎類別確保子類別實作必要的方法。
    """
    
    # 類別變數：帳戶號碼計數器
    _next_account_number = 100001
    
    def __init__(self, initial_balance: float, customer: Customer):
        """初始化銀行帳戶
        
        Args:
            initial_balance: 初始餘額
            customer: 帳戶持有人
        
        Raises:
            InvalidAmountException: 初始餘額無效時拋出
        """
        # TODO: 實作帳戶初始化
        # 提示：生成帳戶號碼、驗證初始餘額、設定客戶關聯
        self._account_number = self._generate_account_number()
        self._balance = validate_amount(initial_balance)
        self._customer = customer
        self._transactions: List[Transaction] = []
        self._created_date = datetime.now()
        self._is_active = True
        
        # 建立客戶關聯
        customer.add_account(self)
        
        # 記錄開戶交易
        if initial_balance > 0:
            self._add_transaction('deposit', initial_balance, '開戶存款')
    
    @classmethod
    def _generate_account_number(cls) -> str:
        """生成唯一的帳戶號碼"""
        # TODO: 實作帳戶號碼生成邏輯
        account_number = str(cls._next_account_number)
        cls._next_account_number += 1
        return account_number
    
    @property
    def account_number(self) -> str:
        """取得帳戶號碼（唯讀）"""
        return self._account_number
    
    @property
    def balance(self) -> float:
        """取得帳戶餘額（唯讀）"""
        return self._balance
    
    @property
    def customer(self) -> Customer:
        """取得帳戶持有人（唯讀）"""
        return self._customer
    
    @property
    def is_active(self) -> bool:
        """檢查帳戶是否啟用"""
        return self._is_active
    
    def deposit(self, amount: float) -> bool:
        """存款操作
        
        Args:
            amount: 存款金額
        
        Returns:
            bool: 操作是否成功
        
        Raises:
            InvalidAmountException: 金額無效時拋出
            AccountClosedException: 帳戶已關閉時拋出
        """
        # TODO: 實作存款邏輯
        # 提示：驗證帳戶狀態、驗證金額、更新餘額、記錄交易
        if not self._is_active:
            raise AccountClosedException(self._account_number)
        
        amount = validate_amount(amount)
        
        # TODO: 完成存款邏輯
        # 更新餘額
        # 記錄交易
        # 回傳成功
        pass
    
    @abstractmethod
    def withdraw(self, amount: float) -> bool:
        """提款操作（抽象方法）
        
        子類別必須實作此方法，因為不同帳戶類型有不同的提款規則。
        
        Args:
            amount: 提款金額
        
        Returns:
            bool: 操作是否成功
        """
        pass
    
    def _basic_withdraw(self, amount: float) -> bool:
        """基礎提款邏輯（供子類別使用）
        
        提供基本的提款驗證與處理，子類別可以在此基礎上加入特殊邏輯。
        """
        # TODO: 實作基礎提款邏輯
        # 提示：驗證帳戶狀態、金額、餘額充足性
        if not self._is_active:
            raise AccountClosedException(self._account_number)
        
        amount = validate_amount(amount)
        
        if amount > self._balance:
            raise InsufficientFundsException(self._balance, amount)
        
        # TODO: 完成提款邏輯
        # 更新餘額
        # 記錄交易
        # 回傳成功
        pass
    
    def _add_transaction(self, transaction_type: str, amount: float, 
                       description: str = "") -> None:
        """新增交易記錄"""
        # TODO: 實作交易記錄新增
        transaction = Transaction(transaction_type, amount, description, self._balance)
        self._transactions.append(transaction)
    
    def get_transaction_history(self, limit: Optional[int] = None) -> List[Transaction]:
        """取得交易記錄
        
        Args:
            limit: 限制回傳的記錄數量
        
        Returns:
            List[Transaction]: 交易記錄列表（按時間倒序）
        """
        # TODO: 實作交易記錄查詢
        # 提示：按時間倒序排列，支援限制數量
        pass
    
    def close_account(self) -> bool:
        """關閉帳戶"""
        # TODO: 實作帳戶關閉邏輯
        # 提示：檢查餘額是否為零、設定狀態
        pass
    
    def __str__(self) -> str:
        """回傳友善的帳戶資訊字串"""
        # TODO: 實作帳戶資訊格式化
        # 提示：包含帳戶號碼、類型、餘額、持有人
        pass
    
    def __repr__(self) -> str:
        """回傳物件的詳細表示"""
        return f"{self.__class__.__name__}(account_number='{self._account_number}', balance={self._balance})"
    
    def __eq__(self, other) -> bool:
        """比較兩個帳戶是否相同（比較帳戶號碼）"""
        # TODO: 實作帳戶比較邏輯
        if not isinstance(other, BankAccount):
            return False
        return self._account_number == other._account_number
    
    # 進階功能：運算子重載
    def __lt__(self, other: 'BankAccount') -> bool:
        """比較帳戶餘額大小"""
        # TODO: 實作餘額比較
        pass
    
    def __le__(self, other: 'BankAccount') -> bool:
        """小於等於比較"""
        # TODO: 實作小於等於比較
        pass
    
    def __gt__(self, other: 'BankAccount') -> bool:
        """大於比較"""
        # TODO: 實作大於比較
        pass
    
    def __ge__(self, other: 'BankAccount') -> bool:
        """大於等於比較"""
        # TODO: 實作大於等於比較
        pass
    
    def __hash__(self):
        """讓帳戶物件可作為字典鍵值"""
        return hash(self._account_number)
    
    def __add__(self, amount: float) -> float:
        """帳戶 + 金額 = 新餘額（模擬存款後餘額）"""
        # TODO: 實作加法運算
        pass
    
    def __sub__(self, amount: float) -> float:
        """帳戶 - 金額 = 新餘額（模擬提款後餘額）"""
        # TODO: 實作減法運算
        pass
    
    def __iadd__(self, amount: float):
        """帳戶 += 金額（實際執行存款）"""
        # TODO: 實作複合賦值加法
        self.deposit(amount)
        return self
    
    def __isub__(self, amount: float):
        """帳戶 -= 金額（實際執行提款）"""
        # TODO: 實作複合賦值減法
        self.withdraw(amount)
        return self

## 💰 SavingsAccount 儲蓄帳戶

In [ ]:
class SavingsAccount(BankAccount):
    """儲蓄帳戶類別
    
    提供利息收入但限制每月提款次數的帳戶類型。
    適合長期儲蓄的客戶使用。
    
    特色：
    - 提供利息計算
    - 限制每月提款次數
    - 無透支功能
    """
    
    def __init__(self, initial_balance: float, customer: Customer, 
                 interest_rate: float = 0.015, withdrawal_limit: int = 6):
        """初始化儲蓄帳戶
        
        Args:
            initial_balance: 初始餘額
            customer: 帳戶持有人
            interest_rate: 年利率（預設 1.5%）
            withdrawal_limit: 每月提款次數限制（預設 6 次）
        """
        # TODO: 實作儲蓄帳戶初始化
        # 提示：呼叫父類別初始化、設定特有屬性
        super().__init__(initial_balance, customer)
        self._interest_rate = interest_rate
        self._withdrawal_limit = withdrawal_limit
        self._monthly_withdrawal_count = 0
        self._last_reset_month = datetime.now().month
    
    @property
    def interest_rate(self) -> float:
        """取得利率（唯讀）"""
        return self._interest_rate
    
    @property
    def withdrawal_limit(self) -> int:
        """取得提款次數限制（唯讀）"""
        return self._withdrawal_limit
    
    @property
    def monthly_withdrawal_count(self) -> int:
        """取得本月提款次數（唯讀）"""
        return self._monthly_withdrawal_count
    
    def withdraw(self, amount: float) -> bool:
        """儲蓄帳戶提款（覆寫父類別方法）
        
        檢查月提款次數限制後執行提款。
        
        Args:
            amount: 提款金額
        
        Returns:
            bool: 操作是否成功
        
        Raises:
            WithdrawalLimitExceededException: 超過月提款次數限制
        """
        # TODO: 實作儲蓄帳戶提款邏輯
        # 提示：先檢查月限制、再執行基本提款、最後更新計數
        self._check_monthly_reset()
        
        if self._monthly_withdrawal_count >= self._withdrawal_limit:
            raise WithdrawalLimitExceededException(
                self._monthly_withdrawal_count, self._withdrawal_limit
            )
        
        # TODO: 完成提款邏輯
        # 執行基本提款
        # 增加提款計數
        # 回傳結果
        pass
    
    def _check_monthly_reset(self) -> None:
        """檢查是否需要重置月提款計數"""
        # TODO: 實作月份檢查與重置邏輯
        current_month = datetime.now().month
        if current_month != self._last_reset_month:
            self.reset_monthly_limit()
    
    def reset_monthly_limit(self) -> None:
        """重置月提款次數（每月呼叫）"""
        # TODO: 實作月限制重置
        self._monthly_withdrawal_count = 0
        self._last_reset_month = datetime.now().month
    
    def calculate_interest(self, days: int = 365) -> float:
        """計算利息
        
        Args:
            days: 計算天數（預設一年）
        
        Returns:
            float: 計算出的利息金額
        """
        # TODO: 實作利息計算
        # 提示：本金 × 利率 × 時間
        return self._balance * self._interest_rate * (days / 365)
    
    def apply_interest(self) -> float:
        """派發利息到帳戶
        
        Returns:
            float: 派發的利息金額
        """
        # TODO: 實作利息派發
        # 提示：計算利息、存入帳戶、記錄交易
        pass
    
    def __str__(self) -> str:
        """回傳儲蓄帳戶資訊字串"""
        # TODO: 實作儲蓄帳戶特有的字串格式
        # 提示：包含利率、提款次數等資訊
        pass

## 💳 CheckingAccount 支票帳戶

In [ ]:
class CheckingAccount(BankAccount):
    """支票帳戶類別
    
    提供透支功能但收取交易手續費的帳戶類型。
    適合日常交易頻繁的客戶使用。
    
    特色：
    - 允許透支（在額度內）
    - 收取交易手續費
    - 無提款次數限制
    """
    
    def __init__(self, initial_balance: float, customer: Customer,
                 overdraft_limit: float = 1000.0, transaction_fee: float = 10.0):
        """初始化支票帳戶
        
        Args:
            initial_balance: 初始餘額
            customer: 帳戶持有人
            overdraft_limit: 透支額度（預設 $1000）
            transaction_fee: 交易手續費（預設 $10）
        """
        # TODO: 實作支票帳戶初始化
        super().__init__(initial_balance, customer)
        self._overdraft_limit = overdraft_limit
        self._transaction_fee = transaction_fee
    
    @property
    def overdraft_limit(self) -> float:
        """取得透支額度（唯讀）"""
        return self._overdraft_limit
    
    @property
    def transaction_fee(self) -> float:
        """取得交易手續費（唯讀）"""
        return self._transaction_fee
    
    def get_available_balance(self) -> float:
        """取得可用餘額（含透支額度）
        
        Returns:
            float: 實際可用的總金額
        """
        # TODO: 實作可用餘額計算
        # 提示：帳戶餘額 + 透支額度
        return self._balance + self._overdraft_limit
    
    def withdraw(self, amount: float) -> bool:
        """支票帳戶提款（覆寫父類別方法）
        
        允許透支但收取手續費。
        
        Args:
            amount: 提款金額
        
        Returns:
            bool: 操作是否成功
        
        Raises:
            InsufficientFundsException: 超過可用餘額時拋出
        """
        # TODO: 實作支票帳戶提款邏輯
        # 提示：檢查可用餘額、扣除手續費、更新餘額、記錄交易
        if not self._is_active:
            raise AccountClosedException(self._account_number)
        
        amount = validate_amount(amount)
        total_amount = amount + self._transaction_fee
        
        # TODO: 完成提款邏輯
        # 檢查可用餘額
        # 更新餘額（包含手續費）
        # 記錄交易
        pass
    
    def charge_fee(self, amount: float) -> None:
        """扣除交易手續費
        
        Args:
            amount: 手續費金額
        """
        # TODO: 實作手續費扣除
        # 提示：直接從餘額扣除、記錄交易
        self._balance -= amount
        self._add_transaction('fee', amount, '交易手續費')
    
    def is_overdrawn(self) -> bool:
        """檢查是否透支
        
        Returns:
            bool: 是否處於透支狀態
        """
        # TODO: 實作透支檢查
        return self._balance < 0
    
    def get_overdraft_amount(self) -> float:
        """取得透支金額
        
        Returns:
            float: 透支金額（負數表示透支）
        """
        # TODO: 實作透支金額計算
        return min(0, self._balance)
    
    def __str__(self) -> str:
        """回傳支票帳戶資訊字串"""
        # TODO: 實作支票帳戶特有的字串格式
        # 提示：包含可用餘額、透支狀態等資訊
        pass

## ⏰ TimeDepositAccount 定期存款帳戶

In [ ]:
class TimeDepositAccount(BankAccount):
    """定期存款帳戶類別
    
    提供較高利率但限制提款時間的帳戶類型。
    適合長期投資的客戶使用。
    
    特色：
    - 較高的利率
    - 固定期限
    - 提前解約需付罰息
    """
    
    def __init__(self, initial_balance: float, customer: Customer,
                 interest_rate: float = 0.025, maturity_months: int = 12,
                 penalty_rate: float = 0.5):
        """初始化定期存款帳戶
        
        Args:
            initial_balance: 初始餘額
            customer: 帳戶持有人
            interest_rate: 年利率（預設 2.5%）
            maturity_months: 到期月數（預設 12 個月）
            penalty_rate: 提前解約罰息率（預設 50%）
        """
        # TODO: 實作定期存款帳戶初始化
        super().__init__(initial_balance, customer)
        self._interest_rate = interest_rate
        self._maturity_months = maturity_months
        self._penalty_rate = penalty_rate
        self._maturity_date = self._created_date + timedelta(days=maturity_months * 30)
    
    @property
    def interest_rate(self) -> float:
        """取得利率（唯讀）"""
        return self._interest_rate
    
    @property
    def maturity_date(self) -> datetime:
        """取得到期日（唯讀）"""
        return self._maturity_date
    
    @property
    def penalty_rate(self) -> float:
        """取得罰息率（唯讀）"""
        return self._penalty_rate
    
    def is_matured(self) -> bool:
        """檢查是否已到期
        
        Returns:
            bool: 是否已到期
        """
        # TODO: 實作到期檢查
        return datetime.now() >= self._maturity_date
    
    def days_to_maturity(self) -> int:
        """計算距離到期的天數
        
        Returns:
            int: 剩餘天數（負數表示已到期）
        """
        # TODO: 實作剩餘天數計算
        delta = self._maturity_date - datetime.now()
        return delta.days
    
    def withdraw(self, amount: float) -> bool:
        """定期存款提款（覆寫父類別方法）
        
        如果未到期，需收取提前解約罰息。
        
        Args:
            amount: 提款金額
        
        Returns:
            bool: 操作是否成功
        """
        # TODO: 實作定期存款提款邏輯
        # 提示：檢查是否到期、計算罰息、執行提款
        if not self._is_active:
            raise AccountClosedException(self._account_number)
        
        amount = validate_amount(amount)
        
        # TODO: 完成提款邏輯
        # 檢查是否到期
        # 如果未到期，計算並扣除罰息
        # 執行提款
        pass
    
    def calculate_maturity_value(self) -> float:
        """計算到期價值
        
        Returns:
            float: 到期時的帳戶總價值
        """
        # TODO: 實作到期價值計算
        # 提示：本金 + 利息
        interest = self._balance * self._interest_rate * (self._maturity_months / 12)
        return self._balance + interest
    
    def early_withdrawal_penalty(self, amount: float) -> float:
        """計算提前解約罰息
        
        Args:
            amount: 提款金額
        
        Returns:
            float: 罰息金額
        """
        # TODO: 實作罰息計算
        # 提示：根據已累積利息計算罰息
        if self.is_matured():
            return 0.0
        
        # 計算已累積利息
        days_elapsed = (datetime.now() - self._created_date).days
        accumulated_interest = self._balance * self._interest_rate * (days_elapsed / 365)
        
        return accumulated_interest * self._penalty_rate
    
    def apply_maturity_interest(self) -> float:
        """到期時派發利息
        
        Returns:
            float: 派發的利息金額
        """
        # TODO: 實作到期利息派發
        if not self.is_matured():
            return 0.0
        
        # TODO: 計算並派發利息
        pass
    
    def __str__(self) -> str:
        """回傳定期存款帳戶資訊字串"""
        # TODO: 實作定期存款特有的字串格式
        # 提示：包含到期日、利率、剩餘天數等資訊
        pass

## 🏛️ Bank 銀行系統類別

In [ ]:
class Bank:
    """銀行系統類別
    
    統一管理所有客戶與帳戶的核心系統。
    提供客戶管理、帳戶創建、轉帳等完整銀行服務。
    """
    
    def __init__(self, bank_name: str):
        """初始化銀行系統
        
        Args:
            bank_name: 銀行名稱
        """
        # TODO: 實作銀行系統初始化
        # 提示：初始化客戶與帳戶字典、設定計數器
        self._bank_name = bank_name
        self._customers: Dict[str, Customer] = {}
        self._accounts: Dict[str, BankAccount] = {}
        self._transaction_counter = 1
        
        # 建立索引以提升查詢效能
        self._accounts_by_customer: Dict[str, List[str]] = {}
        self._accounts_by_type: Dict[str, List[str]] = {
            'savings': [],
            'checking': [],
            'time_deposit': [],
            'credit': []
        }
    
    @property
    def bank_name(self) -> str:
        """取得銀行名稱（唯讀）"""
        return self._bank_name
    
    def create_customer(self, customer_id: str, name: str, 
                       email: str, phone: str) -> Customer:
        """建立新客戶
        
        Args:
            customer_id: 客戶編號
            name: 客戶姓名
            email: 電子郵件
            phone: 電話號碼
        
        Returns:
            Customer: 新建立的客戶物件
        
        Raises:
            DuplicateCustomerException: 客戶ID已存在時拋出
        """
        # TODO: 實作客戶創建邏輯
        # 提示：檢查重複、創建客戶、加入字典、建立索引
        if customer_id in self._customers:
            raise DuplicateCustomerException(customer_id)
        
        # TODO: 完成客戶創建
        # 創建 Customer 物件
        # 加入客戶字典
        # 初始化客戶帳戶索引
        # 回傳客戶物件
        pass
    
    def create_account(self, customer_id: str, account_type: str, 
                      initial_balance: float, **kwargs) -> BankAccount:
        """工廠方法：建立指定類型的帳戶
        
        Args:
            customer_id: 客戶編號
            account_type: 帳戶類型 ('savings', 'checking', 'time_deposit')
            initial_balance: 初始餘額
            **kwargs: 各帳戶類型的特殊參數
        
        Returns:
            BankAccount: 新建立的帳戶物件
        
        Raises:
            CustomerNotFoundException: 客戶不存在時拋出
            ValueError: 不支援的帳戶類型時拋出
        """
        # TODO: 實作帳戶創建邏輯（工廠模式）
        # 提示：查詢客戶、根據類型創建帳戶、註冊到系統
        customer = self.find_customer(customer_id)
        
        # 工廠模式：根據類型創建對應的帳戶
        if account_type == 'savings':
            account = SavingsAccount(initial_balance, customer, **kwargs)
        elif account_type == 'checking':
            account = CheckingAccount(initial_balance, customer, **kwargs)
        elif account_type == 'time_deposit':
            account = TimeDepositAccount(initial_balance, customer, **kwargs)
        else:
            raise ValueError(f"不支援的帳戶類型：{account_type}")
        
        # TODO: 完成帳戶註冊
        # 註冊到帳戶字典
        # 更新索引
        # 回傳帳戶物件
        pass
    
    def find_customer(self, customer_id: str) -> Customer:
        """根據客戶ID查詢客戶
        
        Args:
            customer_id: 客戶編號
        
        Returns:
            Customer: 找到的客戶物件
        
        Raises:
            CustomerNotFoundException: 客戶不存在時拋出
        """
        # TODO: 實作客戶查詢邏輯
        if customer_id not in self._customers:
            raise CustomerNotFoundException(customer_id)
        return self._customers[customer_id]
    
    def find_account(self, account_number: str) -> BankAccount:
        """根據帳戶號碼查詢帳戶
        
        Args:
            account_number: 帳戶號碼
        
        Returns:
            BankAccount: 找到的帳戶物件
        
        Raises:
            ValueError: 帳戶不存在時拋出
        """
        # TODO: 實作帳戶查詢邏輯
        if account_number not in self._accounts:
            raise ValueError(f"帳戶 {account_number} 不存在")
        return self._accounts[account_number]
    
    def transfer_funds(self, from_account: str, to_account: str, 
                      amount: float) -> bool:
        """帳戶間轉帳
        
        Args:
            from_account: 轉出帳戶號碼
            to_account: 轉入帳戶號碼
            amount: 轉帳金額
        
        Returns:
            bool: 轉帳是否成功
        
        Raises:
            SameAccountTransferException: 同帳戶轉帳時拋出
            各種帳戶操作例外
        """
        # TODO: 實作轉帳邏輯
        # 提示：驗證帳戶、檢查同帳戶、執行轉出轉入、記錄交易
        if from_account == to_account:
            raise SameAccountTransferException(from_account)
        
        # TODO: 完成轉帳邏輯
        # 查詢兩個帳戶
        # 執行轉出（withdraw）
        # 執行轉入（deposit）
        # 記錄轉帳交易
        # 回傳成功
        pass
    
    def get_bank_statistics(self) -> Dict:
        """取得銀行統計資訊
        
        Returns:
            Dict: 包含各種統計數據的字典
        """
        # TODO: 實作銀行統計功能
        # 提示：計算客戶數、帳戶數、總存款、各類型帳戶數
        total_deposits = sum(account.balance for account in self._accounts.values())
        
        # TODO: 完成統計計算
        return {
            'bank_name': self._bank_name,
            'total_customers': len(self._customers),
            'total_accounts': len(self._accounts),
            'total_deposits': total_deposits,
            'accounts_by_type': {
                account_type: len(accounts) 
                for account_type, accounts in self._accounts_by_type.items()
            }
        }
    
    def get_customers_by_balance_range(self, min_balance: float, 
                                     max_balance: float) -> List[Customer]:
        """根據餘額範圍查詢客戶"""
        # TODO: 實作餘額範圍查詢
        pass
    
    def get_top_customers(self, limit: int = 10) -> List[Customer]:
        """取得餘額最高的客戶"""
        # TODO: 實作客戶排名功能
        pass
    
    def __str__(self) -> str:
        """回傳銀行資訊字串"""
        # TODO: 實作銀行資訊格式化
        stats = self.get_bank_statistics()
        return f"{self._bank_name} - 客戶: {stats['total_customers']}, 帳戶: {stats['total_accounts']}, 總存款: ${stats['total_deposits']:,.2f}"

## 🚀 進階功能：CreditAccount 信用卡帳戶

In [ ]:
class CreditAccount(BankAccount):
    """信用卡帳戶類別（進階功能）
    
    允許負餘額但有信用額度限制的帳戶類型。
    適合需要信用支付的客戶使用。
    
    特色：
    - 允許負餘額（信用消費）
    - 有信用額度限制
    - 對負餘額收取利息
    - 最低還款金額要求
    """
    
    def __init__(self, initial_balance: float, customer: Customer,
                 credit_limit: float = 5000.0, interest_rate: float = 0.18):
        """初始化信用卡帳戶
        
        Args:
            initial_balance: 初始餘額（通常為0）
            customer: 帳戶持有人
            credit_limit: 信用額度（預設 $5000）
            interest_rate: 年利率（預設 18%）
        """
        # TODO: 實作信用卡帳戶初始化
        super().__init__(initial_balance, customer)
        self._credit_limit = credit_limit
        self._interest_rate = interest_rate
        self._minimum_payment_rate = 0.05  # 最低還款比例 5%
    
    @property
    def credit_limit(self) -> float:
        """取得信用額度（唯讀）"""
        return self._credit_limit
    
    @property
    def available_credit(self) -> float:
        """取得可用信用額度
        
        Returns:
            float: 剩餘可用的信用額度
        """
        # TODO: 實作可用信用額度計算
        # 提示：信用額度 + 當前餘額（負數表示已使用）
        pass
    
    @property
    def outstanding_balance(self) -> float:
        """取得未償還餘額
        
        Returns:
            float: 需要償還的金額（正數）
        """
        # TODO: 實作未償還餘額計算
        return max(0, -self._balance)
    
    def withdraw(self, amount: float) -> bool:
        """信用卡消費（覆寫父類別方法）
        
        允許超支到信用額度限制。
        
        Args:
            amount: 消費金額
        
        Returns:
            bool: 操作是否成功
        """
        # TODO: 實作信用卡消費邏輯
        # 提示：檢查可用信用額度、更新餘額、記錄交易
        pass
    
    def calculate_minimum_payment(self) -> float:
        """計算最低還款金額
        
        Returns:
            float: 本期最低還款金額
        """
        # TODO: 實作最低還款金額計算
        # 提示：未償還餘額 × 最低還款比例
        pass
    
    def calculate_interest_charge(self) -> float:
        """計算利息費用（針對負餘額）
        
        Returns:
            float: 本期利息費用
        """
        # TODO: 實作利息費用計算
        # 提示：只對負餘額收取利息
        pass
    
    def make_payment(self, amount: float) -> bool:
        """信用卡還款
        
        Args:
            amount: 還款金額
        
        Returns:
            bool: 還款是否成功
        """
        # TODO: 實作還款邏輯
        # 提示：增加餘額（朝正數方向）、記錄交易
        return self.deposit(amount)
    
    def __str__(self) -> str:
        """回傳信用卡帳戶資訊字串"""
        # TODO: 實作信用卡特有的字串格式
        # 提示：包含信用額度、可用額度、未償還餘額等
        pass

## 🧪 測試與示範程式碼

In [ ]:
# TODO: 實作完整功能後，執行以下測試程式碼

def test_basic_functionality():
    """測試基本功能"""
    print("=== 測試基本功能 ===")
    
    # 建立銀行系統
    bank = Bank("台灣第一銀行")
    print(f"銀行建立：{bank}")
    
    # 建立客戶
    customer = bank.create_customer(
        customer_id="C001",
        name="王小明",
        email="ming@example.com",
        phone="0912-345-678"
    )
    print(f"客戶建立：{customer}")
    
    # 開立帳戶
    savings = bank.create_account("C001", "savings", 10000.0)
    checking = bank.create_account("C001", "checking", 5000.0)
    
    print(f"儲蓄帳戶：{savings}")
    print(f"支票帳戶：{checking}")
    
    # 測試交易
    savings.deposit(1000.0)
    checking.withdraw(500.0)
    
    print(f"交易後儲蓄帳戶餘額：${savings.balance:,.2f}")
    print(f"交易後支票帳戶餘額：${checking.balance:,.2f}")
    
    # 測試轉帳
    bank.transfer_funds(savings.account_number, checking.account_number, 2000.0)
    
    print(f"轉帳後儲蓄帳戶餘額：${savings.balance:,.2f}")
    print(f"轉帳後支票帳戶餘額：${checking.balance:,.2f}")
    
    # 顯示統計
    stats = bank.get_bank_statistics()
    print(f"銀行統計：{stats}")

def test_advanced_features():
    """測試進階功能"""
    print("\n=== 測試進階功能 ===")
    
    # TODO: 完成進階功能實作後取消註解
    # bank = Bank("進階銀行")
    # customer = bank.create_customer("C002", "李小華", "hua@example.com", "0987-654-321")
    
    # # 測試信用卡帳戶
    # credit = bank.create_account("C002", "credit", 0.0, credit_limit=10000.0)
    # print(f"信用卡帳戶：{credit}")
    
    # # 測試運算子重載
    # savings1 = bank.create_account("C002", "savings", 5000.0)
    # savings2 = bank.create_account("C002", "savings", 3000.0)
    
    # print(f"帳戶比較：{savings1 > savings2}")
    # savings1 += 1000  # 存款
    # print(f"運算子重載後餘額：${savings1.balance:,.2f}")
    
    print("進階功能測試完成（需先實作相關功能）")

def test_error_handling():
    """測試例外處理"""
    print("\n=== 測試例外處理 ===")
    
    bank = Bank("測試銀行")
    customer = bank.create_customer("C003", "張小美", "mei@example.com", "0911-111-111")
    account = bank.create_account("C003", "savings", 1000.0)
    
    # TODO: 測試各種例外情況
    # 提示：餘額不足、無效金額、帳戶不存在等
    
    print("例外處理測試完成")

# 執行測試（實作完成後取消註解）
# test_basic_functionality()
# test_advanced_features()
# test_error_handling()

print("\n🎯 恭喜！您已完成銀行帳戶系統的基礎框架。")
print("📝 請根據 TODO 註解逐步實作各個功能。")
print("🚀 完成後執行測試程式碼驗證功能正確性。")

## 📋 實作檢核清單

### 基本功能 (70分)
- [ ] 完成所有自訂例外類別
- [ ] 完成資料驗證工具函式
- [ ] 完成 Transaction 交易記錄類別
- [ ] 完成 Customer 客戶類別
- [ ] 完成 BankAccount 基礎類別
- [ ] 完成 SavingsAccount 儲蓄帳戶
- [ ] 完成 CheckingAccount 支票帳戶
- [ ] 完成 TimeDepositAccount 定期存款
- [ ] 完成 Bank 銀行系統類別

### 進階功能 (30分)
- [ ] 完成所有特殊方法實作
- [ ] 完成 CreditAccount 信用卡帳戶
- [ ] 完成運算子重載功能
- [ ] 完成進階銀行統計功能

### 測試與品質
- [ ] 所有測試程式碼正常執行
- [ ] 例外處理完整且正確
- [ ] 程式碼符合 PEP 8 規範
- [ ] 所有方法都有適當的文件字串

---

**🎯 完成所有項目即可提交您的專案！**